# Lab 3

**Submission instructions**

Submit the Colab link to your notebook in Canvas. In addition, take screenshots of the map for each question and submit them to Canvas as well.

In [115]:
from datetime import datetime

now = datetime.now()
print(f"Submitted time: {now}")

Submitted time: 2024-03-23 21:35:15.095876


## Readings

- [3.3 Earth Engine data types](https://book.geemap.org/chapters/03_gee_data.html#earth-engine-data-types)
- [3.4. Earth Engine Data Catalog](https://book.geemap.org/chapters/03_gee_data.html#earth-engine-data-catalog)

## Question 1

Find a DEM dataset in the [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/), clip the DEM to your home state, and display the DEM with a proper color palette on the map. For example, the sample map below shows the DEM of the state of Colorado. 

In [116]:
import ee
import geemap

In [117]:
# ee.Authenticate()
# ee.Initialize()

In [172]:
dem_image = ee.Image('USGS/SRTMGL1_003')

In [173]:
m = geemap.Map()

In [174]:
# Draw polygon on map to obtain coordinates for vector of interest
# michigan = m.user_roi
# michigan.getInfo()
michigan = {'geodesic': False,
 'type': 'Polygon',
 'coordinates': [[[-448.720454, 41.32217],
   [-448.720454, 45.961652],
   [-440.147815, 45.961652],
   [-440.147815, 41.32217],
   [-448.720454, 41.32217]]]}
michigan_bbox = michigan["coordinates"][0][2] # y_max, x_min


In [175]:
# Create Michigan polygon
michigan_polygon = ee.Geometry.Polygon(michigan["coordinates"])
polygon_viz = {
    "border": "Red"
}

In [176]:
# Clip DEM image to Michigan polygon
michigan_dem = dem_image.clip(michigan)

In [177]:
# Image min and max for vis parameters
michigan_stats = geemap.image_stats(michigan_dem)
min = michigan_stats.getInfo()["min"]["elevation"]
max = michigan_stats.getInfo()["max"]["elevation"]

In [178]:
viz_params = {
    "min": min,
    "max": max,
    "palette": "dem"
}

In [179]:
text = "Made by Ilse Paniagua"
m.add_text(text, fontsize=20, position="bottomright")

In [180]:
michigan_bbox

[-440.147815, 45.961652]

In [183]:
m.setCenter(michigan_bbox[0], michigan_bbox[1], 5)
m.addLayer(michigan_polygon, polygon_viz)
m.addLayer(michigan_dem, viz_params)
m

Map(bottom=11963.0, center=[45.961652, -440.147815], controls=(WidgetControl(options=['position', 'transparent…

![](https://i.imgur.com/RhUy93b.png)

## Question 2

Use [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/sentinel-2) or [Landsat-9 data](https://developers.google.com/earth-engine/datasets/catalog/landsat-9), create a cloud-free imagery of the year 2021 for your home state. Display the imagery on the map with a proper band combination. For example, the sample map below shows a cloud-free false-color composite of Sentinel-2 imagery of the year 2021 for the state of Colorado. 

In [159]:
sentinel_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") 
michigan_sentinel = sentinel_collection.filterDate("2021-01-01", "2021-12-31").filterBounds(michigan_polygon).filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10).median()

In [160]:
type(michigan_sentinel)

ee.image.Image

In [161]:
# sentinel_stats = geemap.image_stats(michigan_sentinel)
# sentinel_stats.getInfo()

In [184]:
sentinel_viz = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 300,
    'max': 1000,
    'gamma': [0.95, 1.1, 1]
    }

In [186]:
p = geemap.Map()
p.setCenter(michigan_bbox[0], michigan_bbox[1], 5)
p.addLayer(michigan_sentinel, sentinel_viz)
p

Map(center=[45.961652, -440.147815], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

![](https://i.imgur.com/xkxpkS1.png)